<h1><center>Dynamically generated methods with a non-generic signature</center></h1>
<h2><center>A scikit-learn use-case!</center></h2>
<h3><center>Adrin Jalali</center></h3>
<h4><center>@adrin@kolektiva.social</center></h3>
<h4><center>https://github.com/adrinjalali/talks</center></h3>
<h4><center>July 2023</center></h3>

## Me
- PhD in interpretable methods for cancer diagnostics
- ML consulting
- Worked in an algorithmic privacy and fairness team
- Open source
    - `scikit-learn`
    - `fairlearn`
    - `skops`

## Prerequisites
- `f(*args, **kwargs)`
- `setattr`, `getattr`, `hasattr`
- Dunder methods / attributes, e.g. `__init__`, `__str__`, ...
- Inheritance
- Type annotations, `typing.Optional`
- `help(obj)`

## Motivation: Metadata Routing

```py
from sklearn.base import BaseEstimator, TransformerMixin

class MyTransformer(TransformerMixin, BaseEstimator):
    ...
    def fit(self, X, y, sample_weight=None, other_metadata=None):
        ...
        
```

```py
trs = (
    MyTransformer()
    .set_fit_request(sample_weight=True, other_metadata=False)
)
pipe = Pipeline([("transformer", trs)]).fit(
    X, y, sample_weight=sample_weight
)
```

## Requirements
- No change in estimators
- Should work for third party estimators inheriting from `BaseEstimator`
- With a specific signature rather than `set_fit_request(*args, **kwargs)`
- A good docstring

## Steps
- `inspect` to introspect functions
- `inspect.Signature` to read and create a function signature
- a descriptor returning a function
    - set the docstring and signature of the function as needed
- use `__init_subclass__` to set the required descriptors

## `inspect` / `Signature`

In [ ]:
def f(a: int, *args, b=None, **kwargs):
    pass

class A:
    def g(self, r, *args, s=None, **kwargs):
        pass

In [ ]:
import inspect
inspect.isfunction(f)

In [ ]:
inspect.isfunction(A().g)

In [ ]:
inspect.ismethod(A().g)

In [ ]:
inspect.isfunction(A.g)

In [ ]:
inspect.signature(f)

In [ ]:
for pname, param in inspect.signature(f).parameters.items():
    print(pname, param.kind, param.default, param.annotation)

In [ ]:
for pname, param in inspect.signature(A.g).parameters.items():
    print(pname, param.kind, param.default, param.annotation)

In [ ]:
for pname, param in inspect.signature(A().g).parameters.items():
    print(pname, param.kind, param.default, param.annotation)

More on `inspect`: https://docs.python.org/3/library/inspect.html

## Returning a Function

In [ ]:
def add_5(x):
    return x + 5

add_5(4)

In [ ]:
def create_adder(value):
    def f(x):
        return x + value
    return f

add_5 = create_adder(5)
add_5(4)

In [ ]:
def create_adder(value):
    return lambda x: x + value

add_4 = create_adder(4)
add_4(3)

## Change a function's "outfit"

In [ ]:
def f(*args):
    """This text."""
    res = 0
    for x in args:
        res += x
    return res

help(f)

In [ ]:
f(1, 2, 3)

In [ ]:
f.__doc__

In [ ]:
f.__name__

In [ ]:
inspect.signature(f)

In [ ]:
from typing import Optional

f.__name__ = "adder"

params = [
    inspect.Parameter(
        "a",
        inspect.Parameter.POSITIONAL_OR_KEYWORD,
        default=0,
        annotation=Optional[float],
    ),
    inspect.Parameter(
        "b",
        inspect.Parameter.POSITIONAL_OR_KEYWORD,
        default=0,
        annotation=Optional[float],
    ),
]
f.__signature__ = inspect.Signature(
    params,
    return_annotation=float,
)

f.__doc__ = """This function returns the sum of inputs.

Parameters
----------
a : float
    A first floating point value.
    
b : float
    A second floating point value.
    
Returns
-------
float
    What the sum of a and be would be.
"""

In [ ]:
help(f)

In [ ]:
inspect.signature(f)

In [ ]:
f(1, 2, 3)

More on Python's `Signature` object: https://peps.python.org/pep-0362/

## Descriptors

In [ ]:
class Pet:
    def __init__(self, name):
        print("descriptor init")
        self.name = name
        
    def __get__(self, instance, owner_type=None):
        print("in __get__")
        return f"I'm {self.name}!"

print("defined descriptr class")
    
class A:
    pet = Pet("Goose")

print("accessing pet")
A().pet

In [ ]:
class Pet:
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, owner_type=None):
        return f"I'm {self.name} and my owner is {instance.name}!"
    
class A:
    pet = Pet("Goose")
    def __init__(self, name):
        self.name = name
    
A("Adrin").pet

In [ ]:
class RequestMethod:
    def __init__(self, keys):
        # accepted arguments to the function
        self.keys = keys
        
    def __get__(self, instance, owner_type=None):
        def f(**kwargs):
            extra_keys = set(kwargs) - set(self.keys)
            if extra_keys:
                raise TypeError(
                    f"Unexpected arguments: {extra_keys}"
                )
            
            for key, value in kwargs.items():
                setattr(instance, f"request_{key}", value)
                
            return instance
                
        return f
    
class Estimator:
    set_fit_request = RequestMethod(["sample_weight"])


In [ ]:
est = Estimator().set_fit_request(sample_weight=True)
est.request_sample_weight

In [ ]:
est.set_fit_request(param=False)

In [ ]:
help(est.set_fit_request)

In [ ]:
def get_signature(owner, keys):
    params = [
        inspect.Parameter(
            name="self",
            kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
            annotation=owner,
        )
    ]
    params.extend(
        [
            inspect.Parameter(
                k,
                inspect.Parameter.KEYWORD_ONLY,
                default=None,
                annotation=Optional[bool],
            )
            for k in keys
        ]
    )
    return inspect.Signature(
        params,
        return_annotation=owner,
    )
    

In [ ]:
class RequestMethod:
    def __init__(self, keys, method_name):
        self.keys = keys
        self.method_name = method_name
    
    def __get__(self, instance, owner_type=None):
        def f(**kwargs):
            extra_keys = set(kwargs) - set(self.keys)
            if extra_keys:
                raise TypeError(
                    f"Unexpected arguments: {extra_keys}"
                )
            
            for key, value in kwargs.items():
                setattr(instance, f"request_{key}", value)
                
            return instance
        
        f.__name__ = self.method_name
        f.__doc__ = "Custom docstring here with all the info."
        f.__signature__ = get_signature(owner_type, self.keys)
        
        return f
    
class Estimator:
    set_fit_request = RequestMethod(
        ["sample_weight"],
        method_name="set_fit_request"
    )

help(Estimator.set_fit_request)

More on descriptors: https://docs.python.org/3/howto/descriptor.html

## `__init_subclass__`

In [ ]:
class Estimator:
    def fit(self, X, y, sample_weight=None):
        pass
    
    def set_fit_request(self, sample_weight=None):
        # we want this automatically generated
        pass

In [ ]:
class Parent:
    def __init_subclass__(cls, **kwargs):
        print(cls)
        cls.attr = 5
        super().__init_subclass__(**kwargs)

print("defined Parent")

class Child(Parent):
    def __init__(self):
        print("creating instance")

print("Defined Child")
Child().attr

In [ ]:
class Parent:
    def __init_subclass__(cls, **kwargs):
        if (
            hasattr(cls, "fit") 
            and inspect.isfunction(getattr(cls, "fit"))
        ):
            print(inspect.signature(getattr(cls, "fit")))
        super().__init_subclass__(**kwargs)
        
class Estimator(Parent):
    def fit(self, X, y, sample_weight=None):
        pass


In [ ]:
class Parent:
    def __init_subclass__(cls, **kwargs):
        if (
            hasattr(cls, "fit") 
            and inspect.isfunction(getattr(cls, "fit"))
        ):
            method_signature = inspect.signature(
                getattr(cls, "fit")
            )
            params = [
                pname for pname in method_signature.parameters
                if pname not in {"self", "X", "y"}
            ]
            setattr(
                cls,
                "set_fit_request", 
                RequestMethod(
                    keys=params,
                    method_name="set_fit_request"
                )
            )
        super().__init_subclass__(**kwargs)
        
class Estimator(Parent):
    def fit(self, X, y, sample_weight=None):
        pass


In [ ]:
Estimator().set_fit_request(sample_weight=True)

In [ ]:
help(Estimator().set_fit_request)

More on `__init_subclass__`: https://peps.python.org/pep-0487/

## Summary
- `inspect` to introspect functions
- `inspect.Signature` to read and create a function signature
- a descriptor returning a function
    - set the docstring and signature of the function as needed
- used `__init_subclass__` to set the required descriptors

<h1><center>❦ Thank YOU! ❦</center></h1>
<h2><center>Questions?</center></h2>